# Classification

## Objectives

*  Fit and evaluate a classification model to predict if a prospect will churn or not.

## Inputs

* outputs/datasets/collection/TelcoCustomerChurn.csv
* Instructions on which variables to use for data cleaning and feature engineering. They are found in each respective notebook.

## Outputs

* Train set (features and target)
* Test set (features and target)
* Data cleaning and Feature Engineering pipeline
* Modeling pipeline
* Feature importance plot



---

# Change working directory

Change the working directory from its current folder to its parent folder


In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspace/Film_Hit_prediction/jupyter_notebooks'

Make the parent of the current directory the new current directory.


In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspace/Film_Hit_prediction'

---

# Load Data 

Load Data

In [4]:
import pandas as pd
import pickle

# Load the splits for modeling
X_train = pd.read_pickle('/workspace/Film_Hit_prediction/jupyter_notebooks/outputs/engineered/X_train.pkl')
X_test = pd.read_pickle('/workspace/Film_Hit_prediction/jupyter_notebooks/outputs/engineered/X_test.pkl')
y_train = pd.read_pickle('/workspace/Film_Hit_prediction/jupyter_notebooks/outputs/engineered/y_train.pkl')
y_test = pd.read_pickle('/workspace/Film_Hit_prediction/jupyter_notebooks/outputs/engineered/y_test.pkl')


print("Dataset shapes:")
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")


Dataset shapes:
X_train shape: (3284, 270)
y_train shape: (3284,)
X_test shape: (813, 270)
y_test shape: (813,)


---

# Step 2: ML Pipeline with all data

In [5]:
# CLEANING TRANSFORMATIONS
import joblib
import pickle
import pandas as pd

# Load and inspect the encoders and filters
with open('/workspace/Film_Hit_prediction/jupyter_notebooks/outputs/cleaned/encoders_and_filters.pkl', 'rb') as f:
    encoders_and_filters = pickle.load(f)

# Let's see what's in this file
print("Keys in encoders_and_filters:", encoders_and_filters.keys())
print("\nContent structure:", encoders_and_filters)

Keys in encoders_and_filters: dict_keys(['mlb_crew', 'crew_min_appearances', 'crew_frequent_columns', 'important_crew_positions', 'mlb_cast', 'cast_min_appearances', 'frequent_cast_members', 'mlb_countries', 'country_min_appearances', 'frequent_countries', 'mlb_companies', 'company_min_appearances', 'frequent_companies', 'language_encoder', 'genre_columns'])

Content structure: {'mlb_crew': MultiLabelBinarizer(), 'crew_min_appearances': 10, 'crew_frequent_columns': ['crew_Director_Aaron Seltzer', 'crew_Director_Adam McKay', 'crew_Director_Adam Shankman', 'crew_Director_Alejandro González Iñárritu', 'crew_Director_Alex Proyas', 'crew_Director_Alexander Payne', 'crew_Director_Alfonso Cuarón', 'crew_Director_Alfred Hitchcock', 'crew_Director_Andrew Adamson', 'crew_Director_Andrew Niccol', 'crew_Director_Andrzej Bartkowiak', 'crew_Director_Andy Fickman', 'crew_Director_Andy Tennant', 'crew_Director_Ang Lee', 'crew_Director_Anne Fletcher', 'crew_Director_Antoine Fuqua', 'crew_Director_Atom 

In [7]:
# CLEANING TRANSFORMATIONS
import joblib
import pickle
import pandas as pd
import numpy as np


cleaning_pipeline = {}
# Load the encoders and filters from cleaning
with open('/workspace/Film_Hit_prediction/jupyter_notebooks/outputs/cleaned/encoders_and_filters.pkl', 'rb') as f:
    encoders_and_filters = pickle.load(f)

# Extract encoders and filters
mlb_crew = encoders_and_filters['mlb_crew']
crew_min_appearances = encoders_and_filters['crew_min_appearances']
crew_frequent_columns = encoders_and_filters['crew_frequent_columns']
important_crew_positions = encoders_and_filters['important_crew_positions']

mlb_cast = encoders_and_filters['mlb_cast']
cast_min_appearances = encoders_and_filters['cast_min_appearances']
frequent_cast_members = encoders_and_filters['frequent_cast_members']

mlb_countries = encoders_and_filters['mlb_countries']
country_min_appearances = encoders_and_filters['country_min_appearances']
frequent_countries = encoders_and_filters['frequent_countries']

mlb_companies = encoders_and_filters['mlb_companies']
company_min_appearances = encoders_and_filters['company_min_appearances']
frequent_companies = encoders_and_filters['frequent_companies']

language_encoder = encoders_and_filters['language_encoder']
genre_columns = encoders_and_filters['genre_columns']

def clean_data(X_train, X_test=None):
    
    def transform_dataset(X):
        X = X.copy()
        
        # Apply filters first
        # Filter crew by minimum appearances
        def filter_crew(crew_list):
            return [person for person in crew_list if person in crew_frequent_columns]

        # Filter cast by minimum appearances
        def filter_cast(cast_list):
            return [actor for actor in cast_list if actor in frequent_cast_members]

        # Filter countries by minimum appearances
        def filter_countries(countries):
            return [country for country in countries if country in frequent_countries]

        # Filter companies by minimum appearances
        def filter_companies(companies):
            return [company for company in companies if company in frequent_companies]

        # Apply filters
        X['crew'] = X['crew'].apply(filter_crew)
        X['cast'] = X['cast'].apply(filter_cast)
        X['production_countries'] = X['production_countries'].apply(filter_countries)
        X['production_companies'] = X['production_companies'].apply(filter_companies)

        # Then apply encodings
        # Crew
        crew_cols = [f'crew_{i}' for i in crew_frequent_columns]
        X[crew_cols] = mlb_crew.transform(X['crew'])
        
        # Cast
        cast_cols = [f'cast_{i}' for i in frequent_cast_members]
        X[cast_cols] = mlb_cast.transform(X['cast'])
        
        # Countries
        country_cols = [f'country_{i}' for i in frequent_countries]
        X[country_cols] = mlb_countries.transform(X['production_countries'])
        
        # Companies
        company_cols = [f'company_{i}' for i in frequent_companies]
        X[company_cols] = mlb_companies.transform(X['production_companies'])
        
        # Language encoding
        X['language_encoded'] = language_encoder.transform(X['original_language'])
        
        # Apply budget filter
        X = X[X['budget'] > 0]

        # Drop original columns after encoding
        columns_to_drop = ['crew', 'cast', 'production_countries', 
                        'production_companies', 'original_language']
        X = X.drop(columns=columns_to_drop, errors='ignore')
        
        return X
    
    # Transform training data
        X_train_clean = transform_dataset(X_train)
        print(f"Training shape after cleaning: {X_train_clean.shape}")
        print("Applied filters:")
        print(f"- Crew min appearances: {crew_min_appearances}")
        print(f"- Cast min appearances: {cast_min_appearances}")
        print(f"- Country min appearances: {country_min_appearances}")
        print(f"- Company min appearances: {company_min_appearances}")
    
    # Transform test data if provided
        if X_test is not None:
            X_test_clean = transform_dataset(X_test)
            print(f"\nTest shape after cleaning: {X_test_clean.shape}")
        return X_train_clean, X_test_clean
    
    return X_train_clean

# Create the cleaning pipeline dictionary with function and encoders
cleaning_pipeline = {
    'clean_data': clean_data,
    'encoders_and_filters': encoders_and_filters
}

# Save the pipeline
with open('/workspace/Film_Hit_prediction/jupyter_notebooks/outputs/cleaned/cleaning_pipeline.pkl', 'wb') as f:
    pickle.dump(cleaning_pipeline, f)

print("Cleaning pipeline saved successfully!")
   


Cleaning pipeline saved successfully!


Inside the pickled files

In [8]:
import pickle
import pandas as pd
import numpy as np

def inspect_pickle(file_path, name=""):
    print(f"\n{'='*50}")
    print(f"Inspecting: {name}")
    print(f"{'='*50}")
    
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    
    print(f"Type: {type(data)}")
    
    if isinstance(data, pd.DataFrame):
        print("\nDataFrame Info:")
        print(f"Shape: {data.shape}")
        print("\nColumns:")
        for col in data.columns:
            print(f"- {col} ({data[col].dtype})")
        print("\nFirst few rows:")
        print(data.head())
        
    elif isinstance(data, pd.Series):
        print("\nSeries Info:")
        print(f"Length: {len(data)}")
        print(f"dtype: {data.dtype}")
        print("\nFirst few values:")
        print(data.head())
        
    elif isinstance(data, (list, tuple)):
        print(f"\nLength: {len(data)}")
        print("First few elements:")
        print(data[:5])
        
    elif hasattr(data, 'get_params'):  # For sklearn objects
        print("\nParameters:")
        print(data.get_params())
        
    else:
        print("\nContent preview:")
        print(data)

# Now inspect each pickle file
base_path = '/workspace/Film_Hit_prediction/jupyter_notebooks/outputs/engineered'

files_to_inspect = {
    'Feature Scaler': 'feature_scaler.pkl',
    'Transformation Data': 'full_transformation_data.pkl',
    'Top Revenue Actors': 'top_revenue_actors.pkl',
    'Top Revenue Directors': 'top_revenue_directors.pkl',
    'Top Revenue Producers': 'top_revenue_producers.pkl',
    'Top Revenue Writers': 'top_revenue_writers.pkl'
}

for name, file in files_to_inspect.items():
    file_path = f"{base_path}/{file}"
    inspect_pickle(file_path, name)


Inspecting: Feature Scaler
Type: <class 'sklearn.preprocessing._data.StandardScaler'>

Parameters:
{'copy': True, 'with_mean': True, 'with_std': True}

Inspecting: Transformation Data
Type: <class 'dict'>

Content preview:
{'feature_scaler': StandardScaler(), 'encoders_and_filters': {'mlb_crew': MultiLabelBinarizer(), 'crew_min_appearances': 10, 'crew_frequent_columns': ['crew_Director_Aaron Seltzer', 'crew_Director_Adam McKay', 'crew_Director_Adam Shankman', 'crew_Director_Alejandro González Iñárritu', 'crew_Director_Alex Proyas', 'crew_Director_Alexander Payne', 'crew_Director_Alfonso Cuarón', 'crew_Director_Alfred Hitchcock', 'crew_Director_Andrew Adamson', 'crew_Director_Andrew Niccol', 'crew_Director_Andrzej Bartkowiak', 'crew_Director_Andy Fickman', 'crew_Director_Andy Tennant', 'crew_Director_Ang Lee', 'crew_Director_Anne Fletcher', 'crew_Director_Antoine Fuqua', 'crew_Director_Atom Egoyan', 'crew_Director_Barry Levinson', 'crew_Director_Barry Sonnenfeld', 'crew_Director_Ben St

Feature Engineering pieline

In [ ]:
import pandas as pd
import numpy as np
from typing import Dict, List, Any
import pickle

class MovieFeatureEngineeringPipeline:
    def __init__(self, 
                feature_scaler: Any,
                transform_data: Dict[str, Any],
                actor_data: Dict[str, Dict],
                director_data: Dict[str, Dict],
                producer_data: Dict[str, Dict],
                writer_data: Dict[str, Dict]):
        
        self.feature_scaler = feature_scaler
        self.transform_data = transform_data
        self.actor_data = actor_data
        self.director_data = director_data
        self.producer_data = producer_data
        self.writer_data = writer_data
        
    def add_actor_metrics(self, df: pd.DataFrame) -> pd.DataFrame:
        # Get the metrics for each actor
        metrics = self.actor_data['metrics']
        
        # Initialize new feature columns
        df['cast_avg_revenue'] = 0.0
        df['cast_revenue_consistency'] = 0.0
        df['cast_hit_rate'] = 0.0
        df['cast_popularity'] = 0.0
        df['cast_pop_consistency'] = 0.0
        df['cast_rev_pop_correlation'] = 0.0
        df['cast_experience'] = 0.0
        df['top_cast_composite_score'] = 0.0
        
        # Calculate aggregated actor metrics
        cast_columns = [col for col in df.columns if col.startswith('cast_') and col in self.actor_data['columns']]
        
        for idx, row in df.iterrows():
            present_actors = [col.replace('cast_', '') for col in cast_columns if row[col] == 1]
            
            if present_actors:
                actor_metrics = [metrics[actor] for actor in present_actors if actor in metrics]
                if actor_metrics:
                    df.loc[idx, 'cast_avg_revenue'] = np.mean([m['avg_revenue'] for m in actor_metrics])
                    df.loc[idx, 'cast_revenue_consistency'] = np.mean([m['revenue_consistency'] for m in actor_metrics])
                    df.loc[idx, 'cast_hit_rate'] = np.mean([m['hit_rate'] for m in actor_metrics])
                    df.loc[idx, 'cast_popularity'] = np.mean([m['avg_popularity'] for m in actor_metrics])
                    df.loc[idx, 'cast_pop_consistency'] = np.mean([m['popularity_consistency'] for m in actor_metrics])
                    df.loc[idx, 'cast_rev_pop_correlation'] = np.mean([m['revenue_popularity_correlation'] for m in actor_metrics])
                    df.loc[idx, 'cast_experience'] = np.mean([m['movies_count'] for m in actor_metrics])
                    df.loc[idx, 'top_cast_composite_score'] = np.mean([m['composite_score'] for m in actor_metrics])
        
        return df

    def add_director_metrics(self, df: pd.DataFrame) -> pd.DataFrame:
        metrics = self.director_data['metrics']
        
        df['director_avg_revenue'] = 0.0
        df['director_hit_rate'] = 0.0
        df['director_experience'] = 0.0
        df['director_composite_score'] = 0.0
        
        director_columns = [col for col in df.columns if col.startswith('crew_Director_') and col in self.director_data['columns']]
        
        for idx, row in df.iterrows():
            present_directors = [col.replace('crew_Director_', '') for col in director_columns if row[col] == 1]
            
            if present_directors:
                director_metrics = [metrics[director] for director in present_directors if director in metrics]
                if director_metrics:
                    df.loc[idx, 'director_avg_revenue'] = np.mean([m['avg_revenue'] for m in director_metrics])
                    df.loc[idx, 'director_hit_rate'] = np.mean([m['hit_rate'] for m in director_metrics])
                    df.loc[idx, 'director_experience'] = np.mean([m['movies_count'] for m in director_metrics])
                    df.loc[idx, 'director_composite_score'] = np.mean([m['composite_score'] for m in director_metrics])
        
        return df

    def add_producer_metrics(self, df: pd.DataFrame) -> pd.DataFrame:
        metrics = self.producer_data['metrics']
        
        df['producer_avg_revenue'] = 0.0
        df['producer_hit_rate'] = 0.0
        df['producer_experience'] = 0.0
        df['producer_composite_score'] = 0.0
        
        producer_columns = [col for col in df.columns if col.startswith('crew_Producer_') and col in self.producer_data['columns']]
        
        for idx, row in df.iterrows():
            present_producers = [col.replace('crew_Producer_', '') for col in producer_columns if row[col] == 1]
            
            if present_producers:
                producer_metrics = [metrics[producer] for producer in present_producers if producer in metrics]
                if producer_metrics:
                    df.loc[idx, 'producer_avg_revenue'] = np.mean([m['avg_revenue'] for m in producer_metrics])
                    df.loc[idx, 'producer_hit_rate'] = np.mean([m['hit_rate'] for m in producer_metrics])
                    df.loc[idx, 'producer_experience'] = np.mean([m['movies_count'] for m in producer_metrics])
                    df.loc[idx, 'producer_composite_score'] = np.mean([m['composite_score'] for m in producer_metrics])
        
        return df

    def add_writer_metrics(self, df: pd.DataFrame) -> pd.DataFrame:
        metrics = self.writer_data['metrics']
        
        df['writer_avg_revenue'] = 0.0
        df['writer_hit_rate'] = 0.0
        df['writer_experience'] = 0.0
        df['writer_composite_score'] = 0.0
        
        writer_columns = [col for col in df.columns if col.startswith('crew_Writer_') and col in self.writer_data['columns']]
        
        for idx, row in df.iterrows():
            present_writers = [col.replace('crew_Writer_', '') for col in writer_columns if row[col] == 1]
            
            if present_writers:
                writer_metrics = [metrics[writer] for writer in present_writers if writer in metrics]
                if writer_metrics:
                    df.loc[idx, 'writer_avg_revenue'] = np.mean([m['avg_revenue'] for m in writer_metrics])
                    df.loc[idx, 'writer_hit_rate'] = np.mean([m['hit_rate'] for m in writer_metrics])
                    df.loc[idx, 'writer_experience'] = np.mean([m['movies_count'] for m in writer_metrics])
                    df.loc[idx, 'writer_composite_score'] = np.mean([m['composite_score'] for m in writer_metrics])
        
        return df

    def add_combined_metrics(self, df: pd.DataFrame) -> pd.DataFrame:
        # Crew experience metrics
        df['total_crew_experience'] = (
            df['director_experience'] + 
            df['producer_experience'] + 
            df['writer_experience']
        )
        
        # Success metrics
        df['avg_crew_hit_rate'] = (
            df[['director_hit_rate', 'producer_hit_rate', 'writer_hit_rate']].mean(axis=1)
        )
        
        # Composite scores
        df['crew_composite_score'] = (
            df[['director_composite_score', 'producer_composite_score', 
                'writer_composite_score']].mean(axis=1)
        )
        
        # Overall project score
        df['project_score'] = (
            (df['crew_composite_score'] * 0.6) + 
            (df['top_cast_composite_score'] * 0.4)
        )
        
        return df

    def add_budget_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """Add budget-related features"""
        # Budget per minute of runtime
        df['budget_per_minute'] = df['budget'] / df['runtime']
        
        # Budget compared to crew average
        df['budget_to_director_avg'] = df['budget'] / df['director_avg_revenue']
        df['budget_to_producer_avg'] = df['budget'] / df['producer_avg_revenue']
        
        # Fill any infinities or NaNs
        budget_cols = ['budget_per_minute', 'budget_to_director_avg', 'budget_to_producer_avg']
        df[budget_cols] = df[budget_cols].replace([np.inf, -np.inf], np.nan)
        df[budget_cols] = df[budget_cols].fillna(0)
        
        return df

    def scale_features(self, df: pd.DataFrame) -> pd.DataFrame:
        numeric_features = [
            'cast_avg_revenue', 'cast_revenue_consistency', 'cast_hit_rate',
            'cast_popularity', 'cast_pop_consistency', 'cast_rev_pop_correlation',
            'cast_experience', 'top_cast_composite_score',
            'director_avg_revenue', 'director_hit_rate', 'director_experience',
            'director_composite_score', 'producer_avg_revenue', 'producer_hit_rate',
            'producer_experience', 'producer_composite_score', 'writer_avg_revenue',
            'writer_hit_rate', 'writer_experience', 'writer_composite_score',
            'total_crew_experience', 'avg_crew_hit_rate', 'crew_composite_score',
            'project_score', 'budget_per_minute', 'budget_to_director_avg',
            'budget_to_producer_avg'
        ]
        
        df[numeric_features] = self.feature_scaler.transform(df[numeric_features])
        return df

    def transform(self, df: pd.DataFrame) -> pd.DataFrame:

        # Make a copy to avoid modifying the input
        df = df.copy()
        
        # Add metrics for each role
        df = self.add_actor_metrics(df)
        df = self.add_director_metrics(df)
        df = self.add_producer_metrics(df)
        df = self.add_writer_metrics(df)
        
        # Add combined and derived features
        df = self.add_combined_metrics(df)
        df = self.add_budget_features(df)
        
        # Scale the engineered features
        df = self.scale_features(df)
        
        return df


def load_pipeline(
    feature_scaler_path: str,
    transform_data_path: str,
    actor_data_path: str,
    director_data_path: str,
    producer_data_path: str,
    writer_data_path: str
) -> MovieFeatureEngineeringPipeline:

    import pickle
    
    with open(feature_scaler_path, 'rb') as f:
        feature_scaler = pickle.load(f)
    
    with open(transform_data_path, 'rb') as f:
        transform_data = pickle.load(f)
        
    with open(actor_data_path, 'rb') as f:
        actor_data = pickle.load(f)
        
    with open(director_data_path, 'rb') as f:
        director_data = pickle.load(f)
        
    with open(producer_data_path, 'rb') as f:
        producer_data = pickle.load(f)
        
    with open(writer_data_path, 'rb') as f:
        writer_data = pickle.load(f)
    
    return MovieFeatureEngineeringPipeline(
        feature_scaler=feature_scaler,
        transform_data=transform_data,
        actor_data=actor_data,
        director_data=director_data,
        producer_data=producer_data,
        writer_data=writer_data
    )



def create_and_save_pipeline(base_path: str = '/workspace/Film_Hit_prediction/jupyter_notebooks/outputs/') -> None:
   
    # Define paths
    paths = {
        'feature_scaler': f"{base_path}engineered/feature_scaler.pkl",
        'transform_data': f"{base_path}engineered/full_transformation_data.pkl",
        'actor_data': f"{base_path}engineered/top_revenue_actors.pkl",
        'director_data': f"{base_path}engineered/top_revenue_directors.pkl",
        'producer_data': f"{base_path}engineered/top_revenue_producers.pkl",
        'writer_data': f"{base_path}engineered/top_revenue_writers.pkl"
    }
    
    # Load all required components
    pipeline = load_pipeline(
        feature_scaler_path=paths['feature_scaler'],
        transform_data_path=paths['transform_data'],
        actor_data_path=paths['actor_data'],
        director_data_path=paths['director_data'],
        producer_data_path=paths['producer_data'],
        writer_data_path=paths['writer_data']
    )
    
    # Save the pipeline
    pipeline_path = f"{base_path}models/movie_feature_engineering_pipeline.pkl"
    with open(pipeline_path, 'wb') as f:
        pickle.dump(pipeline, f)
    
    print(f"Pipeline successfully saved to {pipeline_path}")

if __name__ == "__main__":
    create_and_save_pipeline()




## ML Pipeline for Modelling and Hyperparameter Optimisation

In [9]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import RobustScaler

# Define models

models = {
    'Linear Regression': {
        'model': LinearRegression(),
        'params': {}
    },
    'Ridge': {
        'model': Ridge(random_state=42),
        'params': {
            'alpha': [0.01, 0.1, 1.0, 10.0, 100.0],
            'solver': ['auto', 'svd', 'cholesky']
        }
    },
    'Lasso': {
        'model': Lasso(random_state=42),  
        'params': {
            'alpha': [0.1 ],  
            'max_iter': [500],
            'tol': [0.1], 
            'selection': ['cyclic']
        }   
    },
    'Random Forest': {
        'model': RandomForestRegressor(random_state=42),
        'params': {
            'n_estimators': [100, 200],
            'max_depth': [10, 20, None],
            'min_samples_split': [2, 5],
            'n_jobs': [-1]
        }
    },
}
 



# Set up and train model

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
import joblib
from sklearn.model_selection import cross_val_score

results = {}
best_models = {}

for name, model_info in models.items():
    print(f"\n{'='*50}")
    print(f"\nTraining {name}...")

    # GridSearchCV performs the hyperparameter optimization
    grid_search = GridSearchCV(
        model_info['model'],
        model_info['params'],
        cv=3,
        scoring='neg_mean_squared_error',
        n_jobs= 1 ,
        verbose=2
    )
    
    # Fit the model
    grid_search.fit(X_train, y_train.ravel())
    best_models[name] = grid_search.best_estimator_


# Evaluate Performance

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
import joblib
from sklearn.model_selection import cross_val_score


for name, model in best_models.items():
    print(f"\nTraining {name}...")

     # Make predictions
    y_pred = model.predict(X_test)
    
    
    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    # Store results
    results[name] = {
        'MSE': mse,
        'RMSE': rmse,
        'R2 Score': r2,
        'Best Parameters': grid_search.best_params_
    }
    
    # Print results
    print(f"\nResults for {name}:")
    print(f"RMSE: {rmse:.2f}")
    print(f"R2 Score: {r2:.2f}")
    print(f"Best Parameters: {grid_search.best_params_}")

# Save Best Performing Model

In [ ]:
import os
import joblib

# 1. First identify the best model
best_model_name = min(results, key=lambda x: results[x]['RMSE'])
best_model = best_models[best_model_name]

print(f"\nBest performing model: {best_model_name}")
print(f"Best model RMSE: {results[best_model_name]['RMSE']:.2f}")

# 2. Set up the save directory
save_dir = '/workspace/Film_Hit_prediction/jupyter_notebooks/outputs/models'
os.makedirs(save_dir, exist_ok=True)  # Create directory if it doesn't exist

# 3. Create the filename with model name and date
from datetime import datetime
current_date = datetime.now().strftime("%Y%m%d")
model_filename = f'film_revenue_model_{best_model_name}_{current_date}.joblib'
full_path = os.path.join(save_dir, model_filename)

# 4. Save the model
joblib.dump(best_model, full_path)

print(f"\nBest model saved as: {model_filename}")
print("Model and scalers saved successfully!")
print("Best model saved successfully!")


## Assess feature importance

In [ ]:
import joblib
import pandas as pd
import matplotlib.pyplot as plt

# Load the saved Random Forest model
model_path = '/workspace/Film_Hit_prediction/jupyter_notebooks/outputs/models/film_revenue_model_Random Forest_20250115.joblib'
loaded_model = joblib.load(model_path)

print("Loaded model type:", type(loaded_model))


# Create a DataFrame for feature importance
feature_importance = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': loaded_model.feature_importances_
})

# Sort features by importance in descending order
feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

# Print top 10 most important features
print("\nTop 10 Most Important Features:")
print(feature_importance.head(10))

# Plot the top 10 feature importance
plt.figure(figsize=(12, 6))
plt.bar(feature_importance['Feature'][:10], feature_importance['Importance'][:10])
plt.xticks(rotation=45, ha='right')
plt.title('Top 10 Feature Importance for Revenue Prediction')
plt.xlabel('Features')
plt.ylabel('Importance Score')
plt.tight_layout()
plt.show()


# Prediction Model


Create a function to make predictions for new movies based on parameters:
-  budget (float): Movie budget in dollars
- language (str): Original language (e.g., 'en' for English)
- genres (list): List of genres (e.g., ['Action', 'Adventure'])
    
 Returns:
- float: Predicted revenue and profit

ADd pipeline model

In [ ]:
import joblib
import pickle
import pandas as pd
import numpy as np
import traceback
from typing import Dict, Any, Optional, List

def predict_movie_revenue(
    budget: float,
    runtime: float,
    genres: List[str],
    language: str,
    production_company: str,
    production_country: str,
    actor1: str,
    actor2: str,
    crew_director: str,
    crew_writer: str,
    crew_producer: str,
    base_path: str = '/workspace/Film_Hit_prediction/jupyter_notebooks/outputs/'
) -> Optional[Dict[str, Any]]:
    try:
        print("Loading models and pipelines...")
        
        # Load the model and pipelines
        model = joblib.load(f'{base_path}models/film_revenue_model_Random Forest_20250115.joblib')
        
        # Load cleaning pipeline components
        with open(f'{base_path}cleaned/encoders_and_filters.pkl', 'rb') as f:
            encoders_and_filters = pickle.load(f)
            
        # Load engineering pipeline
        with open(f'{base_path}models/movie_feature_engineering_pipeline.pkl', 'rb') as f:
            engineering_pipeline = pickle.load(f)

        # Extract encoders and filters
        mlb_crew = encoders_and_filters['mlb_crew']
        crew_frequent_columns = encoders_and_filters['crew_frequent_columns']
        frequent_cast_members = encoders_and_filters['frequent_cast_members']
        frequent_countries = encoders_and_filters['frequent_countries']
        frequent_companies = encoders_and_filters['frequent_companies']
        language_encoder = encoders_and_filters['language_encoder']
        genre_columns = encoders_and_filters['genre_columns']

        # Create initial DataFrame with raw features
        raw_data = {
            'budget': budget,
            'runtime': runtime,
            'original_language': language,
            'genres': genres,
            'cast': [actor1, actor2] if actor1 and actor2 else [actor1] if actor1 else [],
            'crew': [
                f"Director_{crew_director}" if crew_director else None,
                f"Writer_{crew_writer}" if crew_writer else None,
                f"Producer_{crew_producer}" if crew_producer else None
            ],
            'production_companies': [production_company] if production_company else [],
            'production_countries': [production_country] if production_country else []
        }
        
        # Remove None values from crew list
        raw_data['crew'] = [c for c in raw_data['crew'] if c is not None]
        
        # Create DataFrame
        input_df = pd.DataFrame([raw_data])

        # Apply cleaning transformations
        # Filter crew by frequency
        input_df['crew'] = input_df['crew'].apply(
            lambda x: [person for person in x if person in crew_frequent_columns]
        )

        # Filter cast by frequency
        input_df['cast'] = input_df['cast'].apply(
            lambda x: [actor for actor in x if actor in frequent_cast_members]
        )

        # Filter countries and companies
        input_df['production_countries'] = input_df['production_countries'].apply(
            lambda x: [country for country in x if country in frequent_countries]
        )
        input_df['production_companies'] = input_df['production_companies'].apply(
            lambda x: [company for company in x if company in frequent_companies]
        )

        # Apply encodings
        # Crew encoding
        crew_cols = [f'crew_{i}' for i in crew_frequent_columns]
        input_df[crew_cols] = mlb_crew.transform(input_df['crew'])

        # Cast encoding
        cast_cols = [f'cast_{i}' for i in frequent_cast_members]
        input_df[cast_cols] = 0  # Initialize with zeros
        for idx, row in input_df.iterrows():
            for actor in row['cast']:
                col = f'cast_{actor}'
                if col in cast_cols:
                    input_df.at[idx, col] = 1

        # Countries and Companies encoding
        country_cols = [f'country_{i}' for i in frequent_countries]
        company_cols = [f'company_{i}' for i in frequent_companies]
        input_df[country_cols] = 0
        input_df[company_cols] = 0
        
        for idx, row in input_df.iterrows():
            for country in row['production_countries']:
                col = f'country_{country}'
                if col in country_cols:
                    input_df.at[idx, col] = 1
            for company in row['production_companies']:
                col = f'company_{company}'
                if col in company_cols:
                    input_df.at[idx, col] = 1

        # Language encoding
        input_df['language_encoded'] = language_encoder.transform(input_df['original_language'])

        # Genre encoding
        for genre in genre_columns:
            input_df[genre] = 1 if genre in genres else 0

        # Drop original columns
        columns_to_drop = ['crew', 'cast', 'production_countries', 
                          'production_companies', 'original_language', 'genres']
        input_df = input_df.drop(columns=columns_to_drop)

        # Add popularity column (initialized to 0 for new movies)
        input_df['popularity'] = 0

        # Apply feature engineering pipeline
        transformed_df = engineering_pipeline.transform(input_df)

        # Make prediction
        predicted_revenue = model.predict(transformed_df)[0]
        predicted_revenue = max(0, predicted_revenue)

        return {
            'revenue': predicted_revenue,
            'profit': predicted_revenue - budget,
            'roi': ((predicted_revenue - budget) / budget * 100) if budget > 0 else 0,
            'is_profitable': predicted_revenue > budget,
        }

    except Exception as e:
        print(f"Error in prediction: {str(e)}")
        traceback.print_exc()
        return None

if __name__ == "__main__":
    # Save the prediction function
    with open('/workspace/Film_Hit_prediction/jupyter_notebooks/outputs/models/predict_movie_revenue.pkl', 'wb') as f:
        pickle.dump(predict_movie_revenue, f)
    print("Prediction function saved successfully!")

New model

In [29]:
'''
import joblib
import pickle
import pandas as pd
import numpy as np
import traceback

def predict_movie_revenue(budget, runtime, genres, language, production_company, 
                          production_country, actor1, actor2, crew_director, 
                          crew_writer, crew_producer):
    try:
        print("Loading models and data...")
        
        # Load the trained model
        model = joblib.load('/workspace/Film_Hit_prediction/jupyter_notebooks/outputs/models/film_revenue_model_Random Forest_20250115.joblib')
        
        # Load the saved transformation data
        with open('/workspace/Film_Hit_prediction/jupyter_notebooks/outputs/engineered/full_transformation_data.pkl', 'rb') as f:
            transform_data = pickle.load(f)

        # Load the feature scaler separately
        with open('/workspace/Film_Hit_prediction/jupyter_notebooks/outputs/engineered/feature_scaler.pkl', 'rb') as f:
            feature_scaler = pickle.load(f)

        # Extract components from transform_data
        numeric_cols = [col for col in transform_data['numeric_cols'] if col != 'revenue']
        all_features = [col for col in transform_data['all_features'] if col != 'revenue']


        # Initialize features with zero values for all columns
        features = {col: 0 for col in all_features}

        # Process numeric features
        features['budget'] = budget
        features['runtime'] = runtime
        features['budget_per_minute'] = budget / runtime if runtime > 0 else 0
        features['popularity'] = 0 

        # Process genres
        for genre in transform_data['genre_columns']:
            features[genre] = 1 if genre in genres else 0

        # Encode language (assuming LabelEncoder was used)
        language_encoder = transform_data['encoders_and_filters']['language_encoder']
        features['language_encoded'] = language_encoder.transform([language])[0] if language in language_encoder.classes_ else 0

        # Load top actors, directors, writers, and producers data
        with open('/workspace/Film_Hit_prediction/jupyter_notebooks/outputs/engineered/top_revenue_actors.pkl', 'rb') as f:
            actor_data = pickle.load(f)
        with open('/workspace/Film_Hit_prediction/jupyter_notebooks/outputs/engineered/top_revenue_directors.pkl', 'rb') as f:
            director_data = pickle.load(f)
        with open('/workspace/Film_Hit_prediction/jupyter_notebooks/outputs/engineered/top_revenue_writers.pkl', 'rb') as f:
            writer_data = pickle.load(f)
        with open('/workspace/Film_Hit_prediction/jupyter_notebooks/outputs/engineered/top_revenue_producers.pkl', 'rb') as f:
            producer_data = pickle.load(f)

        top_actor_cols = actor_data['columns']
        top_director_cols = director_data['columns']
        top_writer_cols = writer_data['columns']
        top_producer_cols = producer_data['columns']

        # Initialize "other" counts
        other_actor_count = 0
        other_director_count = 0
        other_writer_count = 0
        other_producer_count = 0

        # Process actors
        for actor in [actor1, actor2]:
            actor_col = f"cast_{actor.replace(' ', '_')}"
            if actor_col in top_actor_cols:
                features[actor_col] = 1
        else:
                other_actor_count += 1

        # Process director
        director_col = f"crew_Director_{crew_director.replace(' ', '_')}"
        if director_col in top_director_cols:
            features[director_col] = 1
        else:
            other_director_count += 1

        # Process writer
        writer_col = f"crew_Writer_{crew_writer.replace(' ', '_')}"
        if writer_col in top_writer_cols:
            features[writer_col] = 1
        else:
            other_writer_count += 1

        # Process producer
        producer_col = f"crew_Producer_{crew_producer.replace(' ', '_')}"
        if producer_col in top_producer_cols:
            features[producer_col] = 1
        else:
            other_producer_count += 1

        # Add "other" counts to features
        features['other_actor_count'] = other_actor_count
        features['other_director_count'] = other_director_count
        features['other_writer_count'] = other_writer_count
        features['other_producer_count'] = other_producer_count


        # Extract top companies and countries from encoders_and_filters
        top_companies = transform_data['encoders_and_filters']['frequent_companies']
        top_countries = transform_data['encoders_and_filters']['frequent_countries']

        # Process production company (set to 1 if it's in the top list)
        if production_company in top_companies:
            company_col = f"company_{production_company}"
            if company_col in features:
                features[company_col] = 1

        # Process production country (set to 1 if it's in the top list)
        if production_country in top_countries:
            country_col = f"country_{production_country}"
            if country_col in features:
                features[country_col] = 1

        # Debug before creating pred_df
        print(f"Features created: {list(features.keys())}")
        print(f"Expected features (all_features): {all_features}")

        missing_features = set(all_features) - set(features.keys())
        extra_features = set(features.keys()) - set(all_features)

        print(f"Missing features: {missing_features}")
        print(f"Extra features: {extra_features}")

        # Debug feature set before creating pred_df
        print(f"Features created: {list(features.keys())}")
        print(f"Number of expected features: {len(all_features)}")

        # Create DataFrame from features
        pred_df = pd.DataFrame([features])

        print("\nBefore scaling:")
        numeric_data = pred_df[numeric_cols].copy()
        print("Numeric columns:", numeric_cols)
        print("Values:", numeric_data.iloc[0].to_dict())

        # Scale numeric columns
        pred_df[numeric_cols] = feature_scaler.transform(pred_df[numeric_cols])

        scaled_numeric = feature_scaler.transform(pred_df[numeric_cols])
        print("\nAfter scaling:")
        scaled_df = pd.DataFrame(scaled_numeric, columns=numeric_cols)
        print(scaled_df.iloc[0].to_dict())

        # Ensure correct column order
        pred_df = pred_df[all_features]

        # Debug feature count
        print(f"Final number of features in pred_df: {pred_df.shape[1]}")

        # Make prediction
        raw_prediction = model.predict(pred_df)[0]
        print(f"Raw prediction from model: {raw_prediction}")

       # Don't apply scaling factors - the prediction is already in the right units
        predicted_revenue = raw_prediction  # The model predicts in dollars
        predicted_revenue = max(0, predicted_revenue) 
        
        return {
            'revenue': predicted_revenue,
            'profit': predicted_revenue - budget,
            'roi': ((predicted_revenue - budget) / budget * 100) if budget > 0 else 0,
            'is_profitable': predicted_revenue > budget,
        }

    
    
    except Exception as e:
        print(f"Error in prediction: {str(e)}")
        traceback.print_exc()
        return None

# Save the function
with open('/workspace/Film_Hit_prediction/jupyter_notebooks/outputs/models/predict_movie_revenue.pkl', 'wb') as f:
    pickle.dump(predict_movie_revenue, f)
    '''


Alternative prediction test

In [ ]:
import pandas as pd
import numpy as np
import pickle
import joblib

def test_prediction_function():
    print("Starting prediction test...")
    
    # Test data
    test_movie = {
        'budget': 150000000,           # Budget: $150M
        'runtime': 120,                # Runtime: 2 hours
        'genres': ['Action', 'Adventure', 'Science Fiction'],
        'language': 'en',             # English language
        'production_company': 'Universal Pictures',
        'production_country': 'United States of America',
        'actor1': 'Tom Cruise',
        'actor2': 'Morgan Freeman',
        'crew_director': 'Christopher Nolan',
        'crew_writer': 'Quentin Tarantino',
        'crew_producer': 'Kevin Feige'
    }
    
    print("\nInput Data:")
    for key, value in test_movie.items():
        print(f"{key}: {value}")
    
    try:
        # Make prediction
        test_prediction = predict_movie_revenue(
            budget=test_movie['budget'],
            runtime=test_movie['runtime'],
            genres=test_movie['genres'],
            language=test_movie['language'],
            production_company=test_movie['production_company'],
            production_country=test_movie['production_country'],
            actor1=test_movie['actor1'],
            actor2=test_movie['actor2'],
            crew_director=test_movie['crew_director'],
            crew_writer=test_movie['crew_writer'],
            crew_producer=test_movie['crew_producer']
        )
        
        if test_prediction:
            print("\nPrediction Results:")
            print("-" * 50)
            print(f"Predicted Revenue: ${test_prediction['revenue']:,.2f}")
            print(f"Predicted Profit: ${test_prediction['profit']:,.2f}")
            print(f"ROI: {test_prediction['roi']:.1f}%")
            print(f"Is Profitable: {'Yes' if test_prediction['is_profitable'] else 'No'}")
            print("-" * 50)
            
            # Additional analysis
            if test_prediction['revenue'] > test_movie['budget']:
                profit_margin = (test_prediction['profit'] / test_prediction['revenue']) * 100
                print(f"\nProfit Margin: {profit_margin:.1f}%")
                print(f"Return on Investment: {test_prediction['roi']:.1f}%")
            else:
                loss_percentage = (abs(test_prediction['profit']) / test_movie['budget']) * 100
                print(f"\nLoss Percentage: {loss_percentage:.1f}%")
        else:
            print("\nError: Prediction returned None")
            
    except Exception as e:
        print(f"\nError during prediction: {str(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    test_prediction_function()

Test prediction

In [ ]:
import pandas as pd
import numpy as np
import pickle
import joblib

# Test prediction
test_prediction = predict_movie_revenue(
    budget=150000000,           # Budget: $150M
    runtime=120,                # Runtime: 2 hours
    genres=['Action', 'Adventure', 'Science Fiction'],
    language='en',             # English language
    production_company='Universal Pictures', 
    production_country='United States of America',
    actor1='Tom Cruise',
    actor2='Morgan Freeman',
    crew_director='Christopher Nolan',
    crew_writer='Quentin Tarantino',
    crew_producer='Kevin Feige'
)

print("\nPrediction Results:")
print(f"Predicted Revenue: ${test_prediction['revenue']:,.2f}")
print(f"Predicted Profit: ${test_prediction['profit']:,.2f}")
print(f"ROI: {test_prediction['roi']:.1f}%")
print(f"Is Profitable: {test_prediction['is_profitable']}")

Load model

In [ ]:
import joblib

model_path = "/workspace/Film_Hit_prediction/jupyter_notebooks/outputs/models/"  
model = joblib.load(model_path + 'film_revenue_model_Random Forest_20250115.joblib')
print("Model loaded:", type(model))


Make predictions on test data 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pickle
import matplotlib.pyplot as plt
import seaborn as sns

# Load test and train data
with open('/workspace/Film_Hit_prediction/jupyter_notebooks/outputs/engineered/y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)
    
with open('/workspace/Film_Hit_prediction/jupyter_notebooks/outputs/engineered/y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)

# Load X_test if not already loaded
with open('/workspace/Film_Hit_prediction/jupyter_notebooks/outputs/engineered/X_test.pkl', 'rb') as f:
    X_test = pickle.load(f)

# Get predictions
y_pred = model.predict(X_test)

# Calculate metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("\nModel Evaluation Metrics:")
print(f'Root Mean Squared Error: ${rmse:,.2f}')
print(f'Mean Absolute Error: ${mae:,.2f}')
print(f'R² Score: {r2:.4f}')

# Calculate percentage error
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
print(f'Mean Absolute Percentage Error: {mape:.2f}%')





In [ ]:
# Visualization of predictions vs actual
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Revenue')
plt.ylabel('Predicted Revenue')
plt.title('Predicted vs Actual Revenue')
plt.tight_layout()
plt.show()

# Residual plot
residuals = y_test - y_pred
plt.figure(figsize=(10, 6))
plt.scatter(y_pred, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--')
plt.xlabel('Predicted Revenue')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.tight_layout()
plt.show()

# Distribution of residuals
plt.figure(figsize=(10, 6))
sns.histplot(residuals, kde=True)
plt.xlabel('Residuals')
plt.ylabel('Count')
plt.title('Distribution of Residuals')
plt.tight_layout()
plt.show()

Additional analysis

In [ ]:


# Additional analysis
print("\nResiduals Analysis:")
print(f"Mean of residuals: ${residuals.mean():,.2f}")
print(f"Standard deviation of residuals: ${residuals.std():,.2f}")
print(f"Skewness of residuals: {residuals.skew():.2f}")

# Sample predictions
comparison_df = pd.DataFrame({
    'Actual Revenue': y_test,
    'Predicted Revenue': y_pred,
    'Absolute Error': np.abs(y_test - y_pred),
    'Percentage Error': np.abs((y_test - y_pred) / y_test) * 100
})

print("\nSample Predictions (first 5 movies):")
print(comparison_df.head().to_string())